# 门控循环单元（GRU）
---
门控循环单元（Gated Recurrent Unit, GRU）是一种改进的循环神经网络（RNN）架构，旨在解决传统RNN在处理长序列数据时遇到的梯度消失和梯度爆炸问题。GRU通过引入门控机制来控制信息的流动，从而更有效地捕捉序列中的长期依赖关系。

## 基本结构

GRU的核心思想是通过两个门控单元（更新门和重置门）来控制隐藏状态的更新。这两个门控单元决定了哪些信息应该被保留，哪些信息应该被遗忘。

## 数学表示

假设输入序列为 \( x_1, x_2, \dots, x_T \)，隐藏状态序列为 \( h_1, h_2, \dots, h_T \)。GRU的基本更新公式如下：

1. **更新门（Update Gate）**：
\[
z_t = \sigma(W_z \cdot [h_{t-1}, x_t] + b_z)
\]

2. **重置门（Reset Gate）**：
\[
r_t = \sigma(W_r \cdot [h_{t-1}, x_t] + b_r)
\]

3. **候选隐藏状态（Candidate Hidden State）**：
\[
\tilde{h}_t = \tanh(W_h \cdot [r_t \odot h_{t-1}, x_t] + b_h)
\]

4. **最终隐藏状态（Final Hidden State）**：
\[
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
\]

其中：
- \( z_t \) 是更新门，控制当前隐藏状态中保留多少前一个时间步的信息。
- \( r_t \) 是重置门，控制当前隐藏状态中忽略多少前一个时间步的信息。
- \( \tilde{h}_t \) 是候选隐藏状态，表示在当前时间步可能的隐藏状态。
- \( h_t \) 是最终的隐藏状态。
- \( \sigma \) 是sigmoid激活函数，用于将输入压缩到0到1之间。
- \( \tanh \) 是双曲正切激活函数，用于将输入压缩到-1到1之间。
- \( \odot \) 表示逐元素相乘。
![GRU](https://zh-v2.d2l.ai/_images/gru-3.svg "GRU")
## 特点

1. **简化门控机制**：相比于长短期记忆网络（LSTM），GRU的门控机制更为简单，只包含两个门（更新门和重置门），减少了参数数量，便于训练。
2. **有效捕捉长期依赖**：通过更新门和重置门，GRU能够有效地控制信息的流动，捕捉序列中的长期依赖关系，缓解梯度消失问题。
3. **计算效率高**：由于门控机制的简化，GRU在训练和推理时的计算效率较高，适合处理大规模数据。

## 应用

GRU广泛应用于各种序列建模任务，特别是在自然语言处理（NLP）领域：
- **语言模型**：预测下一个词或字符的概率。
- **机器翻译**：将一种语言的句子翻译成另一种语言。
- **语音识别**：将语音信号转换为文本。
- **时间序列预测**：如股票价格预测、天气预报等。

## 简单代码实现

## 初始化模型参数

In [ ]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return np.random.normal(scale=0.01, size=shape, ctx=device)

    def three():
        return (normal((num_inputs, num_hiddens)),
                normal((num_hiddens, num_hiddens)),
                np.zeros(num_hiddens, ctx=device))

    W_xz, W_hz, b_z = three()  # 更新门参数
    W_xr, W_hr, b_r = three()  # 重置门参数
    W_xh, W_hh, b_h = three()  # 候选隐状态参数
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = np.zeros(num_outputs, ctx=device)
    # 附加梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

## 定义模型

In [ ]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = npx.sigmoid(np.dot(X, W_xz) + np.dot(H, W_hz) + b_z)
        R = npx.sigmoid(np.dot(X, W_xr) + np.dot(H, W_hr) + b_r)
        H_tilda = np.tanh(np.dot(X, W_xh) + np.dot(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = np.dot(H, W_hq) + b_q
        outputs.append(Y)
    return np.concatenate(outputs, axis=0), (H,)